Project 3: Classification
Daniel Arthur

Index:
Business Understanding: Notebook clearly explains the project’s value for helping a specific stakeholder solve a real-world problem.
    
    Introduction explains the real-world problem the project aims to solve
    Introduction identifies stakeholders who could use the project and how they would use it
    Conclusion summarizes implications of the project for the real-world problem and stakeholders 


Data Understanding:  Notebook clearly describes the source and properties of the data to show how useful the data are for solving the problem of interest

      Describe the data sources and explain why the data are suitable for the project
    Present the size of the dataset and descriptive statistics for all features used in the analysis
    Justify the inclusion of features based on their properties and relevance for the project
    Identify any limitations of the data that have implications for the project


Data Preparation: Notebook shows how you prepare your data and explains why by including

    Instructions or code needed to get and prepare the raw data for analysis
    Code comments and text to explain what your data preparation code does
    Valid justifications for why the steps you took are appropriate for the problem you are solving


Modeling: Notebook demonstrates an iterative approach to model-building.

    Runs and interprets a simple, baseline model for comparison
    Introduces new models that improve on prior models and interprets their results
    Explicitly justifies model changes based on the results of prior models and the problem context
    Explicitly describes any improvements found from running new models



Evaluation: Notebook shows how well a final model solves the real-world problem.

    Justifies choice of metrics using context of the real-world problem and consequences of errors
    Identifies one final model based on performance on the chosen metrics with validation data
    Evaluates the performance of the final model using holdout test data
    Discusses implications of the final model evaluation for solving the real-world problem


Code Quality: Code in notebook and related files meets professional standards 
    Code is easy to read, using comments, spacing, variable names, and function docstrings
    All code runs and no code or comments are included that are not needed for the project 
    Code minimizes repetition, using loops, functions, and classes
    Code adapted from others is properly cited with author names and location of the cited material
